In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 10 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'mix_20%&s2_60%','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_mix_20p_s2(60p)_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20p_s2(60p)_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20p_s2(60p)_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
11/11 [==============================] - 4s 391ms/step - loss: 3.6495 - participant_output_loss: 1.4227 - command_output_loss: 2.2206 - command_output_1_loss: 7.9962e-04 - participant_output_1_loss: 0.0054 - participant_output_accuracy: 0.4514 - command_output_accuracy: 0.3514 - command_output_1_accuracy: 0.0057 - participant_output_1_accuracy: 0.0943 - val_loss: 3.4181 - val_participant_output_loss: 1.4038 - val_command_output_loss: 2.0139 - val_command_output_1_loss: 2.7688e-04 - val_participant_output_1_loss: 1.6873e-04 - val_participant_output_accuracy: 0.3389 - val_command_output_accuracy: 0.5506 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1418
Epoch 2/10
11/11 [==============================] - 2s 192ms/step - loss: 2.9315 - participant_output_loss: 1.0199 - command_output_loss: 1.9112 - command_output_1_loss: 2.3149e-04 - participant_output_1_loss: 2.2275e-04 - participant_output_accuracy: 0.6271 - command_output_accuracy: 0.7286

Epoch 3/10
11/11 [==============================] - 2s 189ms/step - loss: 0.9318 - participant_output_loss: 0.1923 - command_output_loss: 0.7394 - command_output_1_loss: 3.1955e-05 - participant_output_1_loss: 3.5662e-05 - participant_output_accuracy: 0.9986 - command_output_accuracy: 0.9957 - command_output_1_accuracy: 0.0500 - participant_output_1_accuracy: 0.1671 - val_loss: 1.1180 - val_participant_output_loss: 0.3249 - val_command_output_loss: 0.7930 - val_command_output_1_loss: 3.1383e-05 - val_participant_output_1_loss: 4.8778e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9595 - val_command_output_1_accuracy: 0.0166 - val_participant_output_1_accuracy: 0.2689
Epoch 4/10
11/11 [==============================] - 2s 190ms/step - loss: 0.8259 - participant_output_loss: 0.1659 - command_output_loss: 0.6600 - command_output_1_loss: 2.9938e-05 - participant_output_1_loss: 3.4858e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9986

Epoch 5/10
11/11 [==============================] - 2s 204ms/step - loss: 2.1517 - participant_output_loss: 0.4704 - command_output_loss: 1.6811 - command_output_1_loss: 1.3821e-04 - participant_output_1_loss: 4.3503e-05 - participant_output_accuracy: 0.9514 - command_output_accuracy: 0.8771 - command_output_1_accuracy: 0.1700 - participant_output_1_accuracy: 0.2029 - val_loss: 2.2211 - val_participant_output_loss: 0.5641 - val_command_output_loss: 1.6569 - val_command_output_1_loss: 1.0803e-04 - val_participant_output_1_loss: 4.3905e-05 - val_participant_output_accuracy: 0.9374 - val_command_output_accuracy: 0.8674 - val_command_output_1_accuracy: 0.0018 - val_participant_output_1_accuracy: 0.1878
Epoch 6/10
11/11 [==============================] - 2s 208ms/step - loss: 1.9618 - participant_output_loss: 0.3861 - command_output_loss: 1.5756 - command_output_1_loss: 6.1308e-05 - participant_output_1_loss: 3.8865e-05 - participant_output_accuracy: 0.9714 - command_output_accuracy: 0.9443

Epoch 7/10
11/11 [==============================] - 2s 176ms/step - loss: 0.6787 - participant_output_loss: 0.0795 - command_output_loss: 0.5992 - command_output_1_loss: 3.9261e-05 - participant_output_1_loss: 2.5925e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0014 - participant_output_1_accuracy: 0.1700 - val_loss: 0.8595 - val_participant_output_loss: 0.1798 - val_command_output_loss: 0.6797 - val_command_output_1_loss: 3.4823e-05 - val_participant_output_1_loss: 3.9544e-05 - val_participant_output_accuracy: 0.9742 - val_command_output_accuracy: 0.9558 - val_command_output_1_accuracy: 0.0018 - val_participant_output_1_accuracy: 0.1547
Epoch 8/10
11/11 [==============================] - 2s 175ms/step - loss: 0.6148 - participant_output_loss: 0.0724 - command_output_loss: 0.5423 - command_output_1_loss: 3.0627e-05 - participant_output_1_loss: 2.5561e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 9/10
11/11 [==============================] - 3s 234ms/step - loss: 1.2722 - participant_output_loss: 0.2810 - command_output_loss: 0.9911 - command_output_1_loss: 5.4498e-05 - participant_output_1_loss: 2.4936e-05 - participant_output_accuracy: 0.9857 - command_output_accuracy: 0.9929 - command_output_1_accuracy: 0.1357 - participant_output_1_accuracy: 0.2086 - val_loss: 1.4135 - val_participant_output_loss: 0.3935 - val_command_output_loss: 1.0200 - val_command_output_1_loss: 4.6329e-05 - val_participant_output_1_loss: 3.0346e-05 - val_participant_output_accuracy: 0.9687 - val_command_output_accuracy: 0.9521 - val_command_output_1_accuracy: 0.0902 - val_participant_output_1_accuracy: 0.1786
Epoch 10/10
11/11 [==============================] - 2s 164ms/step - loss: 1.1317 - participant_output_loss: 0.2454 - command_output_loss: 0.8862 - command_output_1_loss: 4.3994e-05 - participant_output_1_loss: 2.4377e-05 - participant_output_accuracy: 0.9929 - command_output_accuracy: 0.992

...

Epoch 7/10
11/11 [==============================] - 2s 190ms/step - loss: 1.6731 - participant_output_loss: 0.3546 - command_output_loss: 1.3184 - command_output_1_loss: 3.3874e-05 - participant_output_1_loss: 3.4197e-05 - participant_output_accuracy: 0.9686 - command_output_accuracy: 0.9771 - command_output_1_accuracy: 0.0900 - participant_output_1_accuracy: 0.2486 - val_loss: 1.7872 - val_participant_output_loss: 0.4760 - val_command_output_loss: 1.3111 - val_command_output_1_loss: 2.9836e-05 - val_participant_output_1_loss: 4.1234e-05 - val_participant_output_accuracy: 0.9503 - val_command_output_accuracy: 0.9319 - val_command_output_1_accuracy: 0.1215 - val_participant_output_1_accuracy: 0.2873
Epoch 8/10
11/11 [==============================] - 3s 256ms/step - loss: 1.5193 - participant_output_loss: 0.3047 - command_output_loss: 1.2145 - command_output_1_loss: 3.3866e-05 - participant_output_1_loss: 3.2278e-05 - participant_output_accuracy: 0.9843 - command_output_accuracy: 0.9800

Epoch 2/10
11/11 [==============================] - 2s 192ms/step - loss: 3.1216 - participant_output_loss: 0.9847 - command_output_loss: 2.1364 - command_output_1_loss: 4.6188e-04 - participant_output_1_loss: 1.2179e-04 - participant_output_accuracy: 0.6557 - command_output_accuracy: 0.5829 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2186 - val_loss: 3.1441 - val_participant_output_loss: 1.0426 - val_command_output_loss: 2.1011 - val_command_output_1_loss: 3.4536e-04 - val_participant_output_1_loss: 6.7071e-05 - val_participant_output_accuracy: 0.6667 - val_command_output_accuracy: 0.6169 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1473
Epoch 3/10
11/11 [==============================] - 2s 195ms/step - loss: 2.8369 - participant_output_loss: 0.7717 - command_output_loss: 2.0649 - command_output_1_loss: 2.4085e-04 - participant_output_1_loss: 5.7331e-05 - participant_output_accuracy: 0.8257 - command_output_accuracy

Epoch 4/10
11/11 [==============================] - 2s 203ms/step - loss: 1.5204 - participant_output_loss: 0.1421 - command_output_loss: 1.3783 - command_output_1_loss: 2.9126e-05 - participant_output_1_loss: 1.4113e-05 - participant_output_accuracy: 0.9986 - command_output_accuracy: 0.9957 - command_output_1_accuracy: 0.1300 - participant_output_1_accuracy: 0.1914 - val_loss: 1.6815 - val_participant_output_loss: 0.2758 - val_command_output_loss: 1.4057 - val_command_output_1_loss: 4.3086e-05 - val_participant_output_1_loss: 2.0820e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9540 - val_command_output_1_accuracy: 0.0276 - val_participant_output_1_accuracy: 0.2541
Epoch 5/10
11/11 [==============================] - 2s 144ms/step - loss: 1.4447 - participant_output_loss: 0.1331 - command_output_loss: 1.3115 - command_output_1_loss: 3.5213e-05 - participant_output_1_loss: 1.3386e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9986

Epoch 6/10
11/11 [==============================] - 2s 153ms/step - loss: 1.8075 - participant_output_loss: 0.4073 - command_output_loss: 1.4002 - command_output_1_loss: 4.9966e-05 - participant_output_1_loss: 2.1555e-05 - participant_output_accuracy: 0.9457 - command_output_accuracy: 0.9257 - command_output_1_accuracy: 0.3843 - participant_output_1_accuracy: 0.2943 - val_loss: 1.9363 - val_participant_output_loss: 0.5453 - val_command_output_loss: 1.3909 - val_command_output_1_loss: 3.8232e-05 - val_participant_output_1_loss: 2.4357e-05 - val_participant_output_accuracy: 0.9116 - val_command_output_accuracy: 0.8803 - val_command_output_1_accuracy: 0.1529 - val_participant_output_1_accuracy: 0.2707
Epoch 7/10
11/11 [==============================] - 2s 185ms/step - loss: 1.6232 - participant_output_loss: 0.3325 - command_output_loss: 1.2907 - command_output_1_loss: 4.0056e-05 - participant_output_1_loss: 2.0002e-05 - participant_output_accuracy: 0.9800 - command_output_accuracy: 0.9314

Epoch 8/10
11/11 [==============================] - 2s 189ms/step - loss: 0.4854 - participant_output_loss: 0.0576 - command_output_loss: 0.4278 - command_output_1_loss: 2.3275e-05 - participant_output_1_loss: 1.2650e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1843 - participant_output_1_accuracy: 0.1886 - val_loss: 0.6917 - val_participant_output_loss: 0.1656 - val_command_output_loss: 0.5261 - val_command_output_1_loss: 2.3059e-05 - val_participant_output_1_loss: 2.0092e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9669 - val_command_output_1_accuracy: 0.3407 - val_participant_output_1_accuracy: 0.2468
Epoch 9/10
11/11 [==============================] - 2s 145ms/step - loss: 0.4396 - participant_output_loss: 0.0527 - command_output_loss: 0.3869 - command_output_1_loss: 2.3216e-05 - participant_output_1_loss: 1.1950e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

In [27]:
resnet_model = tf.keras.models.load_model('Initial_group_model_mix_20p_s2(60p)_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9926 0.9705


In [28]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


In [29]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

In [30]:
best_index

0